# 构建论文的引用关系，粗颗粒度

## 去掉过长摘要的文章

In [8]:
input_file="/home/sduu39/zHongchang/data/v4_107viewInsAbstract.jsonl"


In [16]:
import networkx as nx
G=nx.Graph()

In [ ]:
import json
count=0
with open(input_file) as f:
    for line_num,line in enumerate(f,1):
        try:
            data=json.loads(line.strip())
            index=data.get("index")
            if index is None:
                print(f"警告：第{line_num}行缺少index字段，已跳过")
                continue
            cited_list=data.get("references")
            if not isinstance(cited_list, list):
                print(f"警告：第{line_num}行的'cited'不是列表，已跳过")
                continue
            G.add_node(index)
            for cited_node in cited_list:
                # print(cited_node)
                G.add_node(cited_node)
                G.add_edge(cited_node,index)
            # print("------")
            count+=1
            if count%20000==0:
                # break
                print(count)
        except json.JSONDecodeError:
            # 处理JSON解析错误（如格式错误的行）
            print(f"警告：第{line_num}行JSON格式错误，已跳过")
        except Exception as e:
            # 捕获其他意外错误
            print(f"警告：第{line_num}行处理失败，错误：{str(e)}")



# 将类别一的论文图保存

In [13]:
paperL1="/home/sduu39/zHongchang/data/label/paperLabel1.jsonl"
import json
import networkx as nx, pickle, gzip
G=nx.Graph()
count=0
with open(paperL1) as f:
    for line_num,line in enumerate(f,1):
        try:
            data=json.loads(line.strip())
            index=data.get("index")
            G.add_node(index)
        except Exception as e:
            print(f"警告：第{line_num}行处理失败，错误：{str(e)}")
with open(paperL1) as f:
    for line_num,line in enumerate(f,1):
        try:
            data=json.loads(line.strip())
            index=data.get("index")
            cited_list=data.get("references")
            for cited_node in cited_list:
                if cited_node in G:
                    G.add_edge(cited_node,index)
        except Exception as e:
            print(f"警告：第{line_num}行处理失败，错误：{str(e)}")

with open("/home/sduu39/zHongchang/data/networkx/paperL1_graph.gpickle", "wb") as f:
    pickle.dump(G, f, protocol=pickle.HIGHEST_PROTOCOL)

## 分析类别1的图的各个属性

In [ ]:
paperL1="/home/sduu39/zHongchang/data/label/paperLabel1.jsonl"
import json
import networkx as nx, pickle, gzip
c=0
G1=nx.Graph()
with open(paperL1) as f:
    for line_num,line in enumerate(f,1):
        try:
            data=json.loads(line.strip())
            index=data.get("index")
            G1.add_node(index)
            c+=1

        except Exception as e:
            # 捕获其他意外错误
            print(f"警告：第{line_num}行处理失败，错误：{str(e)}")
    print(c)

In [9]:
extra_nodes = set(G.nodes()) - set(G1.nodes())
print(f"G 比 G1 多出的节点数：{len(extra_nodes)}")
# for n in list(extra_nodes)[:10]:  # 先看前10个
#     print(n, G.nodes[n])
#根据差别index在p1文件找到对应的venue
s=str(list(extra_nodes)[0])
with open("/home/sduu39/zHongchang/data/v4_data.jsonl") as f:
    for line in f:
        data=json.loads(line.strip())
        if index==s:
            print(data['venue'])

G 比 G1 多出的节点数：51834


In [12]:
paperL1="/home/sduu39/zHongchang/data/label/paperLabel1.jsonl"
import json
count=0
with open(paperL1) as f:
    for line in f:
        data=json.loads(line.strip())
        cited_list=data.get("references")
        if len(cited_list)==0:
            count+=1
    print(f"共有{count}个空引用")

共有50247个空引用


In [14]:
num_iso = sum(1 for _ in nx.isolates(G))
print(num_iso)

47313


In [2]:
import json
import networkx as nx
import pickle
from pathlib import Path

paperL1 = "/home/sduu39/zHongchang/data/label/paperLabel1.jsonl"

# 你希望保留到节点上的属性字段（按需增删）
NODE_FIELDS = [
    "index", "title", "year", "authors", "venue",
    "label", "references"
]
G = nx.Graph()  # 如果你想无向图，用 nx.Graph()
# 第一步：单次扫描，收集所有“合法论文”的节点属性 & 引用列表
valid_nodes = {}          # index -> 属性字典
out_edges = {}            # index -> list(references)

with open(paperL1, "r", encoding="utf-8") as f:
    for line_num, line in enumerate(f, 1):
        try:
            data = json.loads(line.strip())
            index = data.get("index")
            if not index:
                continue  # 没有 index 的直接跳过

            # 取出你关心的字段作为节点属性（过滤掉 None）
            attrs = {k: data.get(k) for k in NODE_FIELDS}

            # 也可以做一些清洗或类型转换
            # 例如：如果 index 是数字字符串，转成 int（可选）
            try:
                attrs["index"] = int(index)
                index_key = attrs["index"]
            except Exception:
                print("---")  # 保持原样

            G.add_node(index_key, **attrs)


        except Exception as e:
            print(f"警告：第{line_num}行处理失败，错误：{str(e)}")

print("123")
# 第二步：构图（只在 valid_nodes 中的节点进行连边）

missing_ref = 0
kept_edges = 0
with open(paperL1, "r", encoding="utf-8") as f:
    for line_num, line in enumerate(f, 1):
        try:
            data = json.loads(line.strip())
            index = data.get("index")
            if index is None:
                continue
            try:
                src = int(index)
            except Exception:
                src = str(index)

            if src not in G:
                continue  # 第1遍未入图的节点直接跳过

            refs = data.get("references") or []
            # 简单去重，避免重复边
            seen = set()
            for r in refs:
                try:
                    dst = int(r)
                except Exception:
                    dst = str(r)
                if dst in seen:
                    continue
                seen.add(dst)

                if dst in G:              # 只连“有效”引用
                    G.add_edge(src, dst)  # A -> B 表示 A 引用 B
                    kept_edges += 1
                else:
                    missing_ref += 1
        except Exception as e:
            print(f"警告：第{line_num}行处理失败，错误：{e}")

print(f"完成：节点 {G.number_of_nodes()}，边 {G.number_of_edges()}（过滤掉无效引用 {missing_ref}，保留 {kept_edges}）")

# 保存（gpickle 会保留所有节点/边属性）
with open("/home/sduu39/zHongchang/data/networkx/paperL1_graph.gpickle", "wb") as f:
    pickle.dump(G, f, protocol=pickle.HIGHEST_PROTOCOL)

123
完成：节点 79839，边 86732（过滤掉无效引用 145665，保留 87700）


In [5]:
print(G.nodes[9579])

{'index': 9579, 'title': 'Compiler Directed Early Register Release.', 'year': 2005, 'authors': ['Timothy M. Jones', "Michael F. P. O'Boyle", 'Jaume Abella', 'Antonio González', 'Oguz Ergin'], 'venue': 'IEEE PACT', 'label': None, 'references': []}


In [6]:
list(G.neighbors(9579))

[437231, 1082973, 1135745, 1248039]

In [7]:
print(G.nodes[437231])

{'index': 437231, 'title': 'Selective writeback: exploiting transient values for energy-efficiency and performance.', 'year': 2006, 'authors': ['Deniz Balkan', 'Joseph J. Sharkey', 'Dmitry Ponomarev', 'Kanad Ghose'], 'venue': 'ISLPED', 'label': None, 'references': [360302, 499395, 499362, 418592, 141343, 343860, 419562, 499064, 499359, 285671, 499058, 418675, 286045, 499155, 252705, 418828, 9579, 499219, 252840, 419070, 499391, 9814]}
